<h1 align="center">Neural Network</h1>

<h2>Imports</h2>

In [177]:
import numpy as np
import copy

<h2>Data</h2>
This stuff is copied directly from the Occupancy Detection example

In [178]:
# load training data
training_data = np.loadtxt('data/trainingdata.txt', usecols=(2,3,4,5,6,7), skiprows=1, delimiter=',')
x_training = training_data[:, :-1]
y_training = training_data[:, -1]

# load testing data
testing_data = np.loadtxt('data/testingdata.txt', usecols=(2,3,4,5,6,7), skiprows=1, delimiter=',')
x_testing = testing_data[:, :-1]
y_testing = testing_data[:, -1]

# feature names and their indexes on the 2D NumPy array
feature_index_names = {0:'Temperature', 1:'Humidity', 2:'Light', 3:'CO2', 4:'Humidity_Ratio'}

# statistics of training and testing datasets
print('training data:')
print('{:_<24s} = {:d}'.format('number of samples', y_training.shape[0]))
print('{:_<24s} = {:d}'.format('number of zeros', np.sum(y_training == 0)))
print('{:_<24s} = {:d}'.format('number of ones', np.sum(y_training == 1)))

print('testing data:')
print('{:_<24s} = {:d}'.format('number of samples', y_testing.shape[0]))
print('{:_<24s} = {:d}'.format('number of zeros', np.sum(y_testing == 0)))
print('{:_<24s} = {:d}'.format('number of ones', np.sum(y_testing == 1)))

training data:
number of samples_______ = 8143
number of zeros_________ = 6414
number of ones__________ = 1729
testing data:
number of samples_______ = 2665
number of zeros_________ = 1693
number of ones__________ = 972


For reasons discussed in the Occupancy Detection lab, we shall only consider the features <b>Light</b> and <b>CO<sub>2</sub></b>

In [179]:
x_training_selected = x_training[:,[2, 3]]
x_testing_selected = x_testing[:,[2, 3]]
print(x_training_selected.shape)
print(x_testing_selected.shape)

(8143, 2)
(2665, 2)


We now normalize the data

In [180]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

x_training_selected = normalized(x_training_selected)
print(x_training_selected)
print("\n\n")
x_testing_selected = normalized(x_testing_selected)
print(x_testing_selected)

[[0.50855837 0.86102752]
 [0.51546614 0.85690995]
 [0.5126364  0.8586058 ]
 ...
 [0.47669095 0.87907095]
 [0.46679766 0.88436415]
 [0.47817743 0.87826325]]



[[0.61557055 0.78808178]
 [0.6054123  0.79591202]
 [0.59693796 0.8022874 ]
 ...
 [0.58734213 0.80933876]
 [0.58641191 0.81001301]
 [0.57890263 0.81539668]]


<h2>Activation Functions</h2>

For simplicity, every single node has the same activation function

In [181]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

<h2>Initialize Parameters</h2>
Here we initialize all parameters

In [182]:
#Number of hidden layers
layer = 2
#Size of input, not including bias
xsize = x_training_selected.shape[1]
#Size of output
osize = 1
#Number of nodes in each layer, not including bias and including input and output
size = [xsize,3,2,osize]
#Weight matrices
weights = []
for i in range(layer+1):
    dim1 = size[i+1]
    dim2 = size[i]+1
    weights.append(np.random.uniform(low=-1, high=1, size=(dim1,dim2)))

for w in weights:
    print(w,"\n\n")

[[-0.91784836  0.91641126  0.19322617]
 [-0.76243397 -0.41321782 -0.13342628]
 [-0.10687742 -0.60858244  0.37608763]] 


[[-0.79064151 -0.25498146 -0.4288715  -0.46414462]
 [ 0.5855728   0.57069911 -0.31438415  0.4307743 ]] 


[[-0.92105482 -0.26882643  0.19359532]] 




<h2>Forward Propagation</h2>

In [183]:
def forwardProp(x,W):
    x = np.array(np.append([1],x))
    #Define output from previous layer
    output = [x]
    for l in range(1,layer+2):        #Iterate through each layer
        #Get weights for this layer
        w = W[l-1]
        #Get s
        s = np.matmul(w,output[l-1])
        #Redefine output
        g = np.zeros((s.shape))
        if(l!=layer+1):
            #Add bias
            g = np.append([1],g)
            for i in range(size[l]):
                g[i+1] = sigmoid(s[i])
            #print("Layer ",l," g = ",g,"\n\n")
        else:
            for i in range(size[l]):
                g[i] = sigmoid(s[i])
        output.append(g)
    return output
    
output = forwardProp(x_training_selected[0],copy.deepcopy(weights))
print(output[-1])

print(weights)

[0.30110989]
[array([[-0.91784836,  0.91641126,  0.19322617],
       [-0.76243397, -0.41321782, -0.13342628],
       [-0.10687742, -0.60858244,  0.37608763]]), array([[-0.79064151, -0.25498146, -0.4288715 , -0.46414462],
       [ 0.5855728 ,  0.57069911, -0.31438415,  0.4307743 ]]), array([[-0.92105482, -0.26882643,  0.19359532]])]


<h2>Back Propagation</h2>

In [189]:
#Test Convergence
def converged(W,Wold,tol):
    diffs = []
    for l in range(layer):
        diffs.append(np.max(np.abs(W[l]-Wold[l])))
    return max(diffs)<tol

#Loss
def loss(y,yh):
    return -(y*np.log(yh)+(1-y)*np.log(1-yh))

#Derivative of Loss
def dLoss(y,yh):
    return -(y/yh -(1-y)/(1-yh))

#Back Propagation - done in place so no need to return

def backProp(X,Y,alpha,W,tol):
    W = np.array(W)
    t = 0
    Wold = copy.deepcopy(W)+np.ones(W.shape)*tol*100
    #while not converged(W,Wold,tol):
    while t<1:
        #Iterate through each data point
        for i in range(X.shape[0]-6000):
            x = X[i]
            y = Y[i]
            #print(x,"\n",y,"\n\n")
            #Apply forward propagation
            output = forwardProp(x,W)
            #Get old
            Wold = copy.deepcopy(W)
            #Output Layer
            dl = dLoss(y,output[layer+1])

            #Update weights
            for l in range(layer,0,-1):
                print("Loss: ",dl)
                print("W[l]: ",W[l])
                
                z = np.matmul(W[l-1],output[l-1])
                z = np.append([1],z)
                print("Z: ",z)
                print("sig: ",sigmoid_derivative(z))
                dlZ = dl*sigmoid_derivative(z)
                print("LossZ: ",dlZ)
                dl = np.dot(W[l],dlZ)
                dlW = (1./output[l].shape[0])*np.dot(dlZ,output[l])
                print("\n\n")
                W[l] = W[l]-alpha*dlW
        print("Iteration ",t+1)
        t+=1
    return W
                
                
            
                
                        
                
new_weights = backProp(x_training_selected,y_training,1,copy.deepcopy(weights),0.00001)
print("\n")
for w in new_weights:
    print(w,"\n")

Loss:  [-3.32104666]
W[l]:  [[-0.92105482 -0.26882643  0.19359532]]
Z:  [ 1.         -1.22951727  0.95664496]
sig:  [0.19661193 0.17506966 0.20051619]
LossZ:  [-0.6529574  -0.5814145  -0.66592362]



Loss:  [0.62878946]
W[l]:  [[-0.79064151 -0.25498146 -0.4288715  -0.46414462]
 [ 0.5855728   0.57069911 -0.31438415  0.4307743 ]]
Z:  [ 1.         -0.2854267  -1.08746305 -0.09255531]
sig:  [0.19661193 0.24497657 0.18854376 0.24946536]
LossZ:  [0.12362751 0.15403869 0.11855433 0.15686119]



Loss:  [-2.0451975]
W[l]:  [[-0.49918551  0.15304288  0.61546463]]
Z:  [ 1.         -1.38781985  0.79812853]
sig:  [0.19661193 0.15985356 0.21406169]
LossZ:  [-0.40211023 -0.3269321  -0.43779844]



Loss:  [-0.11875648]
W[l]:  [[-0.86424648 -0.32858644 -0.50247648 -0.53774959]
 [ 0.51196783  0.49709414 -0.38798913  0.35716933]]
Z:  [ 1.         -0.27989196 -1.08976807 -0.09830783]
sig:  [0.19661193 0.24516701 0.18832822 0.24939694]
LossZ:  [-0.02334894 -0.02911517 -0.0223652  -0.0296175 ]



Loss:  [-1

Loss:  [-17.78190446]
W[l]:  [[52.69978782 53.23544786 53.06155782 53.0262847 ]
 [54.07600212 54.06112843 53.17604517 53.92120362]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-3.49613461 -3.90943513 -3.66056176 -4.36589362]



Loss:  [1.]
W[l]:  [[-90.50717108 -89.85494269 -89.39252094]]
Z:  [  1.         120.05238052 122.23859587]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-17.79478988]
W[l]:  [[54.77686407 55.31252411 55.13863407 55.10336095]
 [56.15307837 56.13820469 55.25312142 55.99827987]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-3.49866804 -3.91226805 -3.66321433 -4.36905731]



Loss:  [1.]
W[l]:  [[-90.57270839 -89.92048    -89.45805825]]
Z:  [  1.         124.59029141 126.77650676]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-17.80

Loss:  [1.]
W[l]:  [[-96.99536488 -96.34313649 -95.88071474]]
Z:  [  1.         585.24572567 587.43194101]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-19.0704462]
W[l]:  [[267.85778341 268.39344345 268.21955341 268.18428029]
 [269.23399771 269.21912403 268.33404076 269.07919921]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-3.7494773  -4.19272708 -3.92581943 -4.68226222]



Loss:  [1.]
W[l]:  [[-97.06090219 -96.4086738  -95.94625205]]
Z:  [  1.         590.10894609 592.29516143]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-19.08333162]
W[l]:  [[270.08537219 270.62103224 270.4471422  270.41186908]
 [271.4615865  271.44671281 270.56162955 271.306788  ]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-3.75201072 -4.19556    -3.92847201 -4.6854259 ]







Loss:  [-20.28167544]
W[l]:  [[483.83003189 484.36569193 484.19180189 484.15652877]
 [485.20624619 485.19137251 484.30628924 485.05144769]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-3.98761942 -4.45902151 -4.17516164 -4.9796487 ]



Loss:  [1.]
W[l]:  [[-103.22140943 -102.56918104 -102.10675929]]
Z:  [1.00000000e+00 1.06192345e+03 1.06410967e+03]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-20.29456086]
W[l]:  [[486.19910245 486.73476249 486.56087246 486.52559934]
 [487.57531676 487.56044307 486.6753598  487.42051826]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-3.99015285 -4.46185443 -4.17781422 -4.98281239]



Loss:  [1.]
W[l]:  [[-103.28694674 -102.63471835 -102.1722966 ]]
Z:  [1.00000000e+00 1.06709884e+03 1.06928506e+03]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0. 

 [719.50039177 719.48551808 718.60043482 719.34559327]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-4.23082839 -4.73098178 -4.42980901 -5.28336255]



Loss:  [1.]
W[l]:  [[-109.5129913  -108.86076291 -108.39834116]]
Z:  [1.00000000e+00 1.57374448e+03 1.57593069e+03]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-21.53156093]
W[l]:  [[720.63774006 721.1734001  720.99951006 720.96423694]
 [722.01395436 721.99908067 721.11399741 721.85915586]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [-4.23336182 -4.7338147  -4.43246159 -5.28652624]



Loss:  [1.]
W[l]:  [[-109.57852861 -108.92630022 -108.46387847]]
Z:  [1.00000000e+00 1.57923531e+03 1.58142153e+03]
sig:  [0.19661193 0.         0.        ]
LossZ:  [0.19661193 0.         0.        ]



Loss:  [-21.54444635]
W[l]:  [[723.15280778 723.68846782 723.514

...

Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:

Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:




Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
s

Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]

Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:  [0.19661193 0.21985469 0.20585881 0.24552452]
LossZ:  [nan nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan]]
Z:  [ 1. nan nan]
sig:  [0.19661193        nan        nan]
LossZ:  [nan nan nan]



Loss:  [nan]
W[l]:  [[nan nan nan nan]
 [nan nan nan nan]]
Z:  [ 1.         -0.72462219 -0.89586025  0.26921021]
sig:

In [185]:
output = forwardProp(x_training_selected[0],new_weights)
print(output[-1])

[nan]
